# Azure AI Foundry Agent Service - Single Agent File Search

Azure AI FoundryのFile Search機能を使用してドキュメント検索型AIエージェントを構築するノートブックです。

## 概要

このノートブックでは、アップロードされたファイルから情報を検索し、質問に答えるエージェント（RAG）を作成します。ECサイトのサポートドキュメントを例に、ファイル検索機能を活用したエージェントの実装方法を学習します。

### 学習内容
1. File Searchツールの設定
2. 検索型エージェントの作成

このエージェントは、サポートセンターやFAQシステム、社内ナレッジベースなどの実用的なアプリケーションに応用できます。

# ライブラリのインポート

必要なPythonライブラリとAzure AI Foundry SDKをインポートします。

In [18]:
import os
import json

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    ListSortOrder,
    FilePurpose,
    FileSearchTool,
)


# 環境変数の取得

Azure AI Foundryへの接続に必要な設定情報を環境変数から取得します。

- `PROJECT_ENDPOINT`: Azure AI Foundryプロジェクトのエンドポイント
- `AZURE_DEPLOYMENT_NAME`: 使用するAIモデルのデプロイメント名

これらの値は事前に.envファイルに設定されている必要があります。

In [19]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

# クライアントの初期化

Azure AI Foundryプロジェクトへの接続とエージェント管理用のクライアントを初期化します。

- `AIProjectClient`: Azure AI Foundryプロジェクトとの接続を管理
- `DefaultAzureCredential`: Azure認証（Azure CLI、Managed Identity等を自動検出）
- `agents_client`: エージェントの作成・管理・実行を行うクライアント

In [20]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# AgentClient の作成
agents_client = project_client.agents

# ユーティリティ関数

エージェントの実行結果を表示するためのヘルパー関数を定義します。

`agent_run_outputs`関数は以下の機能を提供します：
- スレッド内のメッセージ一覧を取得・表示
- 画像コンテンツがある場合は保存・表示
- ツール呼び出し情報の詳細表示（Run Stepsから取得）

## file_search_content_outputs関数
File Search実行時の詳細な検索結果を表示する専用関数です。

In [21]:
def agent_run_outputs(thread_id, agents_client, target_dir="./output_images", show_tool_calls=True, run_id=None):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像・ツール呼び出し）をNotebook上に表示＆画像は保存。
    
    Args:
        thread_id: スレッドID
        agents_client: エージェントクライアント
        target_dir: 画像保存ディレクトリ
        show_tool_calls: ツール呼び出し情報を表示するかどうか
        run_id: 特定のRunのツール呼び出し情報を表示する場合のRun ID
    """
    from IPython.display import Image, display
    from azure.ai.agents.models import RunStepToolCallDetails, RunStepFunctionToolCall, RunStepMessageCreationDetails
    
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    # メッセージの重複防止
    displayed_message_ids = set()
    
    # メッセージの表示
    for message in messages:
        # メッセージの重複チェック
        if message.id in displayed_message_ids:
            continue
        displayed_message_ids.add(message.id)
        
        print(f"\n{'='*60}")
        print(f"MESSAGE ROLE: {message.role.upper()}")
        print(f"MESSAGE ID: {message.id}")
        print(f"{'='*60}")
        
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents") and message.image_contents:
            print(f"\n[IMAGES]")
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                agents_client.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"  Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))
    
    # ツール呼び出し情報の表示（Run Stepsから取得）
    if show_tool_calls and run_id:
        print(f"\n{'='*60}")
        print(f"RUN STEPS INFORMATION (RUN ID: {run_id})")
        print(f"{'='*60}")
        
        try:
            # Run Stepsを取得（デフォルトは新しい順なので、古い順に並び替え）
            run_steps = agents_client.run_steps.list(thread_id=thread_id, run_id=run_id)
            run_steps_list = list(run_steps)
            run_steps_list.reverse()  # 実行順序に並び替え（STEP1から順番に）
            
            # 重複防止のためのセット
            displayed_step_ids = set()
            
            print(f"Total Run Steps: {len(run_steps_list)}")
            
            # 全てのrun stepsを実行順序で表示
            for step_num, run_step in enumerate(run_steps_list, 1):
                # 重複チェック
                if run_step.id in displayed_step_ids:
                    print(f"[STEP {step_num}] - SKIPPED (Duplicate Step ID: {run_step.id})")
                    continue
                displayed_step_ids.add(run_step.id)
                
                print(f"\n[STEP {step_num}] - {run_step.type}")
                print(f"  Step ID: {run_step.id}")
                print(f"  Status: {run_step.status}")
                
                # Message Creation Step
                if isinstance(run_step.step_details, RunStepMessageCreationDetails):
                    print(f"  Message Creation Step")
                    if hasattr(run_step.step_details.message_creation, 'message_id'):
                        print(f"  Message ID: {run_step.step_details.message_creation.message_id}")
                
                # Tool Calls Step
                elif isinstance(run_step.step_details, RunStepToolCallDetails):
                    print(f"  Tool Calls Step - {len(run_step.step_details.tool_calls)} tool(s)")
                    
                    for tool_num, tool_call in enumerate(run_step.step_details.tool_calls, 1):
                        print(f"\n    [TOOL CALL {tool_num}]")
                        print(f"    Tool Type: {tool_call.type}")
                        print(f"    Tool Call ID: {tool_call.id}")
                        
                        # Function Tool Call の詳細
                        if isinstance(tool_call, RunStepFunctionToolCall):
                            print(f"    Function Name: {tool_call.function.name}")
                            print(f"    Function Arguments: {tool_call.function.arguments}")
                            # 関数の実行結果を表示（利用可能な場合）
                            if hasattr(tool_call.function, 'output') and tool_call.function.output:
                                print(f"    Function Output: {tool_call.function.output}")
                            elif hasattr(tool_call.function, 'outputs') and tool_call.function.outputs:
                                print(f"    Function Outputs: {tool_call.function.outputs}")
                            elif hasattr(tool_call.function, 'result') and tool_call.function.result:
                                print(f"    Function Result: {tool_call.function.result}")
                        
                        print(f"    {'-'*30}")
                
                # その他のステップタイプ
                else:
                    print(f"  Step Type: {type(run_step.step_details).__name__}")
                
                print(f"  Created At: {run_step.created_at}")
                if hasattr(run_step, 'completed_at') and run_step.completed_at:
                    print(f"  Completed At: {run_step.completed_at}")
                
                print(f"  {'='*50}")
                
        except Exception as e:
            print(f"Error retrieving run steps: {e}")
            print(f"Run ID: {run_id}, Thread ID: {thread_id}")

In [22]:
def file_search_content_outputs(agents_client, thread_id, run_id):
    """
    指定したスレッドID・Run IDのファイル検索ツールによる検索結果コンテンツをNotebook等に出力
    """
    from azure.ai.agents.models import RunAdditionalFieldList, RunStepToolCallDetails, RunStepFileSearchToolCall

    for run_step in agents_client.run_steps.list(
        thread_id=thread_id,
        run_id=run_id,
        include=[RunAdditionalFieldList.FILE_SEARCH_CONTENTS]
    ):
        if isinstance(run_step.step_details, RunStepToolCallDetails):
            for tool_call in run_step.step_details.tool_calls:
                if (
                    isinstance(tool_call, RunStepFileSearchToolCall)
                    and getattr(tool_call, "file_search", None)
                    and getattr(tool_call.file_search, "results", None)
                    and tool_call.file_search.results
                    and getattr(tool_call.file_search.results[0], "content", None)
                    and tool_call.file_search.results[0].content
                    and getattr(tool_call.file_search.results[0].content[0], "text", None)
                ):
                    file_name = tool_call.file_search.results[0].file_name
                    content_text = tool_call.file_search.results[0].content[0].text
                    print(f"参照：{file_name}")
                    print("=" * 60)
                    print(content_text)
                    print("=" * 60)

# ツールの作成

File Search機能を実現するために必要なコンポーネントを作成します。

## File Searchツールの重要な特徴

### 自動最適化された検索処理
File Searchツールは以下の検索ベストプラクティスを自動実装します：
1. **クエリ最適化**: ユーザーの質問を検索用に自動で書き換え
2. **並列検索**: 複雑な質問を複数の検索に分解して並列実行
3. **ハイブリッド検索**: キーワード検索とセマンティック検索の両方を実行
4. **検索結果ランキング**: 最も関連性の高い結果を選択して最終回答を生成

### デフォルト設定値
- **チャンクサイズ**: 800トークン
- **チャンクオーバーラップ**: 400トークン
- **埋め込みモデル**: text-embedding-3-large（256次元）
- **最大チャンク数**: 20個（コンテキストに追加される）

### ファイル制限
- **最大ファイルサイズ**: 512 MB
- **最大トークン数**: ファイルあたり5,000,000トークン
- **ベクトルストア容量**: 最大10,000ファイル

## ベクトルストアの作成

ドキュメントファイルをアップロードし、検索可能なベクトルストアを構築します。

In [23]:
# アップロード対象ディレクトリ
service_docs_dir = "../../infra/sample_data/to_vector_store/service_docs"

# ファイルIDのリスト
file_ids = []

# ディレクトリ内の全ファイルをループしてアップロード
for fname in os.listdir(service_docs_dir):
    file_path = os.path.join(service_docs_dir, fname)
    if os.path.isfile(file_path):
        uploaded_file = agents_client.files.upload_and_poll(file_path=file_path, purpose=FilePurpose.AGENTS)
        print(f"Uploaded file: {fname}, file ID: {uploaded_file.id}")
        file_ids.append(uploaded_file.id)

if not file_ids:
    raise RuntimeError("No files were uploaded. Please check the directory.")

# アップロードした全ファイルを使ってベクトルストアを作成
vector_store = agents_client.vector_stores.create_and_poll(file_ids=file_ids, name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")


Uploaded file: セットアップサポートのパンフレット.pdf, file ID: assistant-LCzf8jq8uDWZFx2KvGXcP4
Uploaded file: パソコン保障プラン.html, file ID: assistant-JVWnycF19E19SCqRucyrHf
Uploaded file: 出荷日ポリシー.pdf, file ID: assistant-DU1oqhK1kESCQT5W8zHbb8
Created vector store, vector store ID: vs_YILWtQ0y19yNNqLQKwDERUt7


In [24]:
# インポートしたファイル数の確認
vector_store.as_dict()
vector_store.file_counts

{'in_progress': 0, 'completed': 3, 'failed': 0, 'cancelled': 0, 'total': 3}

## File Search ツールの定義

作成したベクトルストアを検索するためのツールを定義します。

In [25]:
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

# エージェントの作成

Foundry エージェントを作成します。

In [26]:
file_search_agent  = agents_client.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="service_docs_search_agent",
    instructions=(
        "あなたはECサイトのサポートエージェントです。"
        "アップロードしたファイルから情報を検索（file_search）できます。"
        "ユーザーの質問に、必ずfile_searchを実行して回答を生成してください。"

    ),
    description=(
        "アップロードしたファイルから情報を検索できるエージェントです。"    
    ),
    tools=file_search.definitions,  
    tool_resources=file_search.resources,
    temperature=0.1
)
print(f"Created Agent. AGENT_ID: {file_search_agent.id}")


Created Agent. AGENT_ID: asst_h2z83KnXkPVywrpAJxgNWRpu


In [27]:
agent_dict = file_search_agent.as_dict()
print(json.dumps(agent_dict, indent=2, ensure_ascii=False))

{
  "id": "asst_h2z83KnXkPVywrpAJxgNWRpu",
  "object": "assistant",
  "created_at": 1753813561,
  "name": "service_docs_search_agent",
  "description": "アップロードしたファイルから情報を検索できるエージェントです。",
  "model": "gpt-4.1",
  "instructions": "あなたはECサイトのサポートエージェントです。アップロードしたファイルから情報を検索（file_search）できます。ユーザーの質問に、必ずfile_searchを実行して回答を生成してください。",
  "tools": [
    {
      "type": "file_search"
    }
  ],
  "top_p": 1.0,
  "temperature": 0.1,
  "tool_resources": {
    "file_search": {
      "vector_store_ids": [
        "vs_YILWtQ0y19yNNqLQKwDERUt7"
      ]
    }
  },
  "metadata": {},
  "response_format": "auto"
}


# スレッドの作成

エージェントとの対話を管理するスレッドを作成します。

In [28]:
# Thread の作成
thread = agents_client.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_29otHx8EihYcXYH9q1oIx0gm


# ユーザーメッセージの追加

スレッドにユーザーからの質問やリクエストを追加します。

In [29]:
# メッセージの追加
user_message_1 = "パソコンの保障プランでもっとも安いプランは？"

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message_1,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_3VzRQ8dgD3SR0lwUu4RRgeVo


# Run の実行

エージェントにタスクを実行させ、結果を取得します。

### 1 ターン目

In [30]:
run_1 = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=file_search_agent.id
)

if run_1.status == "failed":
    print(f"Run failed: {run_1.last_error}")
else:
    agent_run_outputs(thread.id, agents_client, show_tool_calls=True, run_id=run_1.id)
    


MESSAGE ROLE: USER
MESSAGE ID: msg_3VzRQ8dgD3SR0lwUu4RRgeVo
パソコンの保障プランでもっとも安いプランは？

MESSAGE ROLE: ASSISTANT
MESSAGE ID: msg_V4uWbFMq8WWg0VNJM3SOM3nT
パソコンの保障プランでもっとも安いプランは「1年プラン」で、料金は税込10,000円です。内容としては、修理費用全額カバー、代替機の提供（最短3営業日）、論理障害によるデータ復旧、24時間365日のサポートなどが含まれています【4:3†パソコン保障プラン.html】。

RUN STEPS INFORMATION (RUN ID: run_jM1DLR3SRs9UdVhggSaly8XB)
Total Run Steps: 2

[STEP 1] - RunStepType.TOOL_CALLS
  Step ID: step_joyMWb7ylU0awiTotcW7BOQM
  Status: RunStepStatus.COMPLETED
  Tool Calls Step - 1 tool(s)

    [TOOL CALL 1]
    Tool Type: file_search
    Tool Call ID: call_6MPjzknSLzcambnZCy4DQQAN
    ------------------------------
  Created At: 2025-07-29 18:26:04+00:00
  Completed At: 2025-07-29 18:26:06+00:00

[STEP 2] - RunStepType.MESSAGE_CREATION
  Step ID: step_0TDLbLglRRWTU3RgWjEsToNx
  Status: RunStepStatus.COMPLETED
  Message Creation Step
  Message ID: msg_V4uWbFMq8WWg0VNJM3SOM3nT
  Created At: 2025-07-29 18:26:06+00:00
  Completed At: 2025-07-29 18:26:07+00:00


In [31]:
# 参照したファイルのコンテンツを出力
file_search_content_outputs(agents_client, thread.id, run_1.id)

参照：パソコン保障プラン.html
この保障プランに加入することで、パソコンの故障やトラブルに対して安心してご利用いただけます。</p>
    </div>
    <div class="content-section">
        <h2>対象となるパソコン</h2>
        <ul>
            <li>Microsoft Surface シリーズ（Surface Pro/Go/Laptop/Book/Studioなど）全モデル</li>
            <li>Microsoft ブランドのデスクトップ・ノートPC</li>
            <li>個人・法人ともにご利用可能です</li>
        </ul>
    </div>
    <div class="content-section">
        <h2>保障プランの内容</h2>
        <ul>
            <li>故障時の修理費用をプラン内で全額カバー（一部例外あり）</li>
            <li>修理期間中は代替機を最短3営業日でご提供</li>
            <li>データ復旧サービス：論理障害によるデータ損失に対応（成功率約90%）</li>
            <li>専用サポート窓口による24時間365日電話・Web対応</li>
        </ul>
    </div>
    <div class="content-section">
        <h2>保障の対象外項目</h2>
        <ul>
            <li>水没・火災・落下等による損傷</li>
            <li>盗難・紛失・自然災害による破損</li>
            <li>ウイルス感染・ソフトウェア起因のトラブル</li>
            <li>お客様自身による改造、分解、過失・故意による破損</li>
            <li>物理障害によるデータ復旧</li>
        </ul>
    </div>
    <div class="content-section">
        <h2>保障プランの価格</h2>
   

### 2 ターン目

In [32]:
# メッセージの追加
user_message_2 = "PCを購入した際、セットアップサポートを受けたいのですがどんなサービスですか？"

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message_2,
)

run_2 = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=file_search_agent.id
)

if run_2.status == "failed":
    print(f"Run failed: {run_2.last_error}")
else:
    agent_run_outputs(thread.id, agents_client, show_tool_calls=True, run_id=run_2.id)


MESSAGE ROLE: USER
MESSAGE ID: msg_3VzRQ8dgD3SR0lwUu4RRgeVo
パソコンの保障プランでもっとも安いプランは？

MESSAGE ROLE: ASSISTANT
MESSAGE ID: msg_V4uWbFMq8WWg0VNJM3SOM3nT
パソコンの保障プランでもっとも安いプランは「1年プラン」で、料金は税込10,000円です。内容としては、修理費用全額カバー、代替機の提供（最短3営業日）、論理障害によるデータ復旧、24時間365日のサポートなどが含まれています【4:3†パソコン保障プラン.html】。

MESSAGE ROLE: USER
MESSAGE ID: msg_G7JOvfK8JcnNRsqLZ3CD5viC
PCを購入した際、セットアップサポートを受けたいのですがどんなサービスですか？

MESSAGE ROLE: ASSISTANT
MESSAGE ID: msg_MNK6rDR8ZP0LknEVIqsWgKKe
PC購入時のセットアップサポートサービスは、主に以下の内容が含まれています（所要時間：約30～60分）：

1. 電源の接続と起動確認
2. Wi-Fi・インターネット接続設定
3. メールアカウントやアプリの初期登録
4. セキュリティ対策（ウイルス対策ソフトの確認・導入）
5. 基本操作レクチャー（マウス、キーボード、タッチ操作）
6. よく使うアプリ（Office・Zoomなど）のインストールサポート

初心者や設定に不安がある方、家族に頼らず自分で使いたい方におすすめです。出張対応・オンライン対応が選べて、初回サポートは無料（キャンペーン中）となっています。ご希望に応じてマニュアルもプレゼントされます【8:0†セットアップサポートのパンフレット.pdf】。

RUN STEPS INFORMATION (RUN ID: run_TFJ1TajU1SvucuTZYNV1uyPo)
Total Run Steps: 2

[STEP 1] - RunStepType.TOOL_CALLS
  Step ID: step_IYbrCeLjOXQoSrOiyzx1f0zr
  Status: RunStepStatus.COMPLETED
  Tool Calls Step - 

In [33]:
# 参照したファイルのコンテンツを出力
file_search_content_outputs(agents_client, thread.id, run_2.id)

参照：セットアップサポートのパンフレット.pdf
初心者向けセットアップサポート
～パソコン・スマホの初期設定を安心サポート～

サービス内容（所要時間：約30～60分）
1.電源の接続と起動確認
2.Wi-Fi・インターネット接続設定
3.メールアカウント・アプリの初期登録
4.セキュリティ対策（ウイルス対策ソフトの確認・導入）
5.基本操作レクチャー（マウス、キーボード、タッチ操作）
6.よく使うアプリのインストールサポート（Office・Zoomなど）

こんな方におすすめ！
•初めてパソコンやスマホを購入した方
•設定がよくわからず不安な方
•家族に頼らず自分で使いたい方

カスタマーサポート
ご不明点やご予約は下記までお気軽に！
電話：0120-123-456（平日 10:00～18:00）
メール：support@easysetup.jp
WEB予約：https://www.easysetup.jp

安心のサポート体制
出張対応／オンライン対応選べます
初回サポート無料（キャンペーン中）
ご希望に応じてマニュアルもプレゼント！

https://www.easysetup.jp/

	スライド 1: 🎉初心者向けセットアップサポート🎉 ～パソコン・スマホの初期設定を安心サポート～


### 3 ターン目

In [34]:
# メッセージの追加
user_message_3 = "PCを購入した際、どれくらいで出荷されますか？"

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message_3,
)

run_3 = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=file_search_agent.id
)

if run_3.status == "failed":
    print(f"Run failed: {run_3.last_error}")
else:
    agent_run_outputs(thread.id, agents_client, show_tool_calls=True, run_id=run_3.id)


MESSAGE ROLE: USER
MESSAGE ID: msg_3VzRQ8dgD3SR0lwUu4RRgeVo
パソコンの保障プランでもっとも安いプランは？

MESSAGE ROLE: ASSISTANT
MESSAGE ID: msg_V4uWbFMq8WWg0VNJM3SOM3nT
パソコンの保障プランでもっとも安いプランは「1年プラン」で、料金は税込10,000円です。内容としては、修理費用全額カバー、代替機の提供（最短3営業日）、論理障害によるデータ復旧、24時間365日のサポートなどが含まれています【4:3†パソコン保障プラン.html】。

MESSAGE ROLE: USER
MESSAGE ID: msg_G7JOvfK8JcnNRsqLZ3CD5viC
PCを購入した際、セットアップサポートを受けたいのですがどんなサービスですか？

MESSAGE ROLE: ASSISTANT
MESSAGE ID: msg_MNK6rDR8ZP0LknEVIqsWgKKe
PC購入時のセットアップサポートサービスは、主に以下の内容が含まれています（所要時間：約30～60分）：

1. 電源の接続と起動確認
2. Wi-Fi・インターネット接続設定
3. メールアカウントやアプリの初期登録
4. セキュリティ対策（ウイルス対策ソフトの確認・導入）
5. 基本操作レクチャー（マウス、キーボード、タッチ操作）
6. よく使うアプリ（Office・Zoomなど）のインストールサポート

初心者や設定に不安がある方、家族に頼らず自分で使いたい方におすすめです。出張対応・オンライン対応が選べて、初回サポートは無料（キャンペーン中）となっています。ご希望に応じてマニュアルもプレゼントされます【8:0†セットアップサポートのパンフレット.pdf】。

MESSAGE ROLE: USER
MESSAGE ID: msg_XsmnlS9jtcFMh3d5Akb3gUqV
PCを購入した際、どれくらいで出荷されますか？

MESSAGE ROLE: ASSISTANT
MESSAGE ID: msg_cIEl0PaLVYCuYR5umyr3mhbl
PCを購入した際の出荷日は、「受注確認日から7営業日以内」に発送されます。なお、設置工事が必要な商品やメーカーからの取り

In [35]:
# 参照したファイルのコンテンツを出力
file_search_content_outputs(agents_client, thread.id, run_3.id)

参照：出荷日ポリシー.pdf
2025-07-28

1 / 2

出荷⽇ポリシー
当ECサイトでは、お客様に安⼼してショッピングをお楽しみいただくため、商品カテゴリに応じた出荷⽇ポ
リシーを設けております。ご注⽂いただいた商品は、以下のポリシーに基づき順次発送されます。

1. デジタルコンテンツ・ダウンロード商品
出荷⽇︓ご購⼊後、即時ダウンロード可能
詳細︓決済完了と同時に、ダウンロードリンクが記載されたメールが⾃動送信されます。物理的な商
品の発送はありません。

2. 書籍・CD/DVD・ゲームソフト
出荷⽇︓受注確認⽇〜3営業⽇以内
詳細︓在庫がある場合、注⽂後の変更受付⽇以降に出荷⼿配を開始します。新刊・予約商品は、発売
⽇以降の出荷となります。

3. ファッション・アパレル・雑貨
出荷⽇︓受注確認⽇〜5営業⽇以内
詳細︓在庫状況や注⽂内容により、出荷までお時間をいただく場合があります。お⽀払い⽅法の確認
後、発送準備となります。
※注意事項︓アパレル・雑貨の中でもキャラクターグッズについては、別途「キャラクターグッズ出
荷⽇ポリシー」をご確認ください。

4. PC・家電・家具・⼤型商品
出荷⽇︓受注確認⽇〜7営業⽇以内
詳細︓設置⼯事が必要な商品や、メーカーからの取り寄せが必要な商品は、上記以上の納期をいただ
く場合があります。別途、配送業者からの連絡が⼊る場合があります。


# トレースの確認

[Azure AI Foundry Portal](https://ai.azure.com/?cid=learnDocs) でエージェントの実行トレースを確認してみましょう。

# Agent ID を .env ファイルに保存

作成したFile SearchエージェントのIDを永続化し、他のノートブックで再利用できるようにします。

In [36]:
# 変数の定義
agent_env_key = "FOUNDRY_FILE_SEARCH_AGENT_ID"
agent_env_value = file_search_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv()  # 見つからなければ''を返す
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')


.envファイルに FOUNDRY_FILE_SEARCH_AGENT_ID="asst_h2z83KnXkPVywrpAJxgNWRpu" を追記しました。
